# get Cause - efffect phrase from the sentence.

In [1]:
#filtered_rows.reset_index(drop=True, inplace=True)


## 1. load cause-effect data

In [2]:
import ast
import pandas as pd
#'data/in/semeval/semeval_causalityDetection.csv'
#'data/in/SCITE/SCITE_TrueFalse.csv'
#'data/in/RAG/ADE/ADE_causalityTEST.csv'
#load semeval data with embed

#'data/in/semeval/sem_CE_test.csv'
#'data/in/RAG/ADE/ADE_CE_extract_TEST.csv')
#'data/in/RAG/c_e_Extraction/SCITE_CE_extract_INPUT.csv'


pm_DB = pd.read_csv('data/in/KNN_RAG/SCITE_CE_extract_INPUT.csv')
instance_emb = pm_DB['embedings'].to_list()
instance_emb = [ast.literal_eval(item) for item in instance_emb]
instances = pm_DB['sentence'].to_list()
len(instances)

191

## 2. load few shot database

In [3]:
#load fewshotDB with embed
semeval_data = pd.read_csv('data/in/KNN_RAG/comb_SCITEsemADE_CausalityPattern.csv')
fewshot_emb = semeval_data['embedings'].to_list()
fewshot_emb = [ast.literal_eval(item) for item in fewshot_emb]

examples = semeval_data['sentence'].to_list()
len(examples)

2365

## 3. identify cause- effect phrase

In [4]:
#load causality input sentence.
#input_sentence_list = filtered_rows['sentence'].to_list()
#gpt to extract cause effect and a causality phrase.
#: few shot: example1: , causality phrase: .

In [5]:
import groq
from groq import Groq
client = groq.Groq(api_key= "enter your API key here..")


def C_E_CP_Identifier(causality_sentence, fewshot_examples):

    chat_completion = client.chat.completions.create(
    #
    # Required parameters
    #
    messages=[
        # Set an optional system message. This sets the behavior of the
        # assistant and can be used to provide specific instructions for
        # how it should behave throughout the conversation.
        {
          "role": "system",
          "content": '''
  
          You will be given a sentence that shows a causality relation. 
          Your task is to identify all the cause phrase, effect phrase, and causality phrase from
          that sentence. Below is the hints and rules for the task.

          Hints: In sentences showing a causality relation, you can identify a pair of phrases
          where one phrase causes the other (effect). There will also be a phrase between the
          cause and effect phrases that indicates the causality relation; this is the 
          causality phrase. And in most sentences causality phrase can be seen between the 
          cause and effect phrase.
           

          Your output should be in JSON format with keys cause, effect, and causality_phrase:
          {"cause1": "", "effect1": "", "causality_phrase1": "", "cause2": "", "effect2": "", "causality_phrase2": "",
          "cause3": "", "effect3": "", "causality_phrase3": "", ....}. Make sure your JSON ouput is closed with } at the end.
          
          Rule 1: If you are not able to find the cause phrase, effect phrase, and causality phrase 
          from the given sentence you can output empty string for the keys in the json format 
          and do not guess the answer. 
          
          Rule 2: Ensure that there are no overlapping phrases between the cause phrase, effect phrase,
          and causality phrase. For instance, consider the input sentence "Fever is caused by virus." 
          Your identified output without overlap should be:
          {"cause": "virus", "effect": "fever", "causality_phrase": "caused by"}
          Your output should not contain overlap, as shown below:
          {"cause": "by virus", "effect": "fever is caused", "causality_phrase": "caused by"}
          This means that each phrase (cause, effect, and causality phrase) should be distinct 
          and not overlap with each other.
          
          Rule 3: There could be multiple cause-effect phrases in the sentence. You should extract all 
          of them. For instance, consider the input sentence "Mutations in MCFD2 causing F5F8D are null
          mutations but also missense mutations, which cause destabilization of its flexible structure
          and the loss of its interaction with ERGIC-53 .". 
          Your identified output will be:
          {"cause1": "Mutations in MCFD2", "effect1": "F5F8D", "causality_phrase1": "causing", 
          "cause2": "missense mutations", "effect2": "destabilization of its flexible structure", "causality_phrase2": "cause",
          "cause3": "missense mutations", "effect3": "the loss of its interaction with ERGIC-53", "causality_phrase3": "cause"

          
          Below are 10 example sentences that show causality relations, with their cause and
          effect phrases delimited between the XML tags <cause>, </cause>, <effect>, and 
          </effect>. The corresponding causality phrases are also provided.
          

                   
          ''' + fewshot_examples
            
        },
        {
          "role": "user",
          "content": input_sentence 
        },
    ],

    # The language model which will generate the completion.
    model="mixtral-8x7b-32768",
    temperature=0.0,
    seed = 1,
    max_tokens=100,
    top_p=1,
    response_format={"type": "json_object"},)
    out = chat_completion.choices[0].message.content
    
    return out

In [6]:
import json
import re

def close_json_string(json_string):
    
    
    # Remove entries where "cause", "effect", and "causality_phrase" are all empty
    pattern = re.compile(r'"cause\d+":\s*"",\s*"effect\d+":\s*"",\s*"causality_phrase\d+":\s*""\s*,?\s*')
    json_string = re.sub(pattern, '', json_string)
    
    # Remove incomplete index entries
    incomplete_index_pattern = re.compile(r'"cause\d+":\s*""\s*,?\s*')
    json_string = re.sub(incomplete_index_pattern, '', json_string)
    
    # Remove trailing comma if exists
    json_string = re.sub(r',\s*}$', '}', json_string.strip())
    json_string = re.sub(r',\s*$', '', json_string.strip())
    
    
    
    # Find the last complete index
    last_index = re.findall(r'"cause\d+":\s*"[^"]*",\s*"effect\d+":\s*"[^"]*",\s*"causality_phrase\d+":\s*"[^"]*"', json_string)
    
    # If found, truncate the JSON string after the last complete index
    if last_index:
        last_index_end_pos = json_string.rfind(last_index[-1]) + len(last_index[-1])
        json_string = json_string[:last_index_end_pos]
    
    # Count the number of opening and closing braces
    open_braces = json_string.count('{')
    close_braces = json_string.count('}')
    
    # Add missing closing braces if needed
    if open_braces > close_braces:
        json_string += '}' * (open_braces - close_braces)
    
    return json_string

In [7]:
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
filtered_rows = pm_DB

sent_cause_effect_list = []
causality_phrase_list = []

fewshotfound_inputsentence_index=[]
for i in range(len(filtered_rows)):
    input_sentence = filtered_rows['sentence'][i]
    #input_sentence = 'fever is caused by virus'
    
    #print('input_sentence', input_sentence)
    
    # few shot examples augmentation.
    ##########KNN###########
    # Combine lists into a numpy array
    lists = np.array(fewshot_emb)

    # Calculate cosine similarity between list_1 and other lists
    similarities = cosine_similarity([instance_emb[i]], fewshot_emb).flatten()

    # Get indices of the two most similar lists
    most_similar_indices = similarities.argsort()[-10:][::-1]

    # Print the most similar lists and their cosine similarity values

    fewshot_examples = [examples[j] for j in most_similar_indices]

    fewshot_examples = {f'example {k + 1}': sentence for k, sentence in enumerate(fewshot_examples)}
    fewshot_examples = json.dumps(fewshot_examples)   
    ##############print('fewshot_examples', fewshot_examples)

    #fewshot_examples = get_fewshotexamples_C_E_CP(fewshot_index) 
    
    #print('fewshotfound_inputsentence_index', fewshotfound_inputsentence_index)
    try:
    
        # output generation
        out = C_E_CP_Identifier(input_sentence, fewshot_examples)
        #print('out1 : ', out)

        out = close_json_string(out)
        print('out : ', out)

    
    # Attempt to parse the JSON string into a dictionary
        data = json.loads(out)
    except json.JSONDecodeError as e:
        print("Failed to parse JSON:", e)
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])
        continue  # Skip to the next iteration if there is a JSON error

        
    except TypeError as e:
        print('TypeError')
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])
        continue
        
    except KeyError as e:
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])
        continue
    
    except groq.APIConnectionError as e:
        print('APIConnectionError')
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])
        continue
        
    except groq.RateLimitError as e:
        print('RateLimitError')
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])    
        continue
        
    except groq.APIStatusError as e:
        print('APIStatusError')
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])
        continue
    # Parse JSON string into a dictionary
    data = json.loads(out)

    # Initialize lists
    sent_cause_effect = []
    causality_phrase_row = []

    # Iterate through the keys and collect values
    i_out = 1
    while f'cause{i_out}' in data and f'effect{i_out}' in data and f'causality_phrase{i_out}' in data:
        sent_cause_effect.append((input_sentence, data[f'cause{i_out}'], data[f'effect{i_out}']))
        causality_phrase_row.append(data[f'causality_phrase{i_out}'])
        i_out += 1

    if sent_cause_effect == []:
        print('empty')
        try:
            sent_cause_effect.append((input_sentence, data['cause'], data['effect']))
            causality_phrase_row.append(data['causality_phrase'])
        except KeyError as e:
            sent_cause_effect_list.append([])
            causality_phrase_list.append([]) 
            continue
        
        
        
    # Print the results
    print("sent_cause_effect:", sent_cause_effect)
    #print("causality_phrase_row:", causality_phrase_row)
    sent_cause_effect_list.append(sent_cause_effect)
    causality_phrase_list.append(causality_phrase_row)
    print('i:', i)
    
    #if i==3:
        #break;
    

out :  {
  "cause1": "using solar electricity instead of conventional electricity sources",
  "effect1": "reduction of pollution",
  "causality_phrase1": "helps to reduce",
  "cause2": "pollution",
  "effect2": "smog, acid rain, respiratory illness and global warming",
  "causality_phrase2": "that causes"}
sent_cause_effect: [('Using solar electricity instead of conventional electricity sources helps to reduce  pollution  that causes  smog ,  acid rain ,  respiratory illness  and  global warming .', 'using solar electricity instead of conventional electricity sources', 'reduction of pollution'), ('Using solar electricity instead of conventional electricity sources helps to reduce  pollution  that causes  smog ,  acid rain ,  respiratory illness  and  global warming .', 'pollution', 'smog, acid rain, respiratory illness and global warming')]
i: 0
out :  {
  "cause": "capital murders",
  "effect": "execution",
  "causality_phrase": "that result in"
}
empty
sent_cause_effect: [('Thus, eva

out :  {
  "cause": "the construction",
  "effect": "dust, noise, and sleep deprivation produced",
  "causality_phrase": "produced by"
}
empty
sent_cause_effect: [(' The dust ,  noise , and  sleep deprivation  produced by  the construction  are intolerable.', 'the construction', 'dust, noise, and sleep deprivation produced')]
i: 25
out :  {
  "cause1": "the sun",
  "effect1": "irritation",
  "causality_phrase1": "from",
  "cause2": "environmental pollutants",
  "effect2": "inflammation",
  "causality_phrase2": "that helps reduce"}
sent_cause_effect: [('Oak root and chamomile provide a gentle astringent that helps reduce inflammation and calms  skin irritation  from  the sun  and  environmental pollutants .', 'the sun', 'irritation'), ('Oak root and chamomile provide a gentle astringent that helps reduce inflammation and calms  skin irritation  from  the sun  and  environmental pollutants .', 'environmental pollutants', 'inflammation')]
i: 26
out :  {
  "cause1": "deliberately planted f

out :  {
  "cause": "staring effect",
  "effect": "Eye discomfort",
  "causality_phrase": "from this is exacerbated by"
}
empty
sent_cause_effect: [(' Eye discomfort  from  this staring effect  is exacerbated by low humidity.', 'staring effect', 'Eye discomfort')]
i: 49
out :  {"cause": "loss of key employees", "effect": "big risk", "causality_phrase": "from"}
empty
sent_cause_effect: [('Companies face  big risk  from  loss  of key employees.', 'loss of key employees', 'big risk')]
i: 50
out :  {
  "cause": "the coup",
  "effect": "the confusion",
  "causality_phrase": "caused by"
}
empty
sent_cause_effect: [('Also, in the midst of  the confusion  caused by  the coup , the issuing of militant posters was a way for artists and organizations to declare their active engagement against the rebellion.', 'the coup', 'the confusion')]
i: 51
out :  {
  "cause1": "individual donors",
  "effect1": "a significant difference in addressing the suffering",
  "causality_phrase1": "make",
  "cause2": 

out :  {
  "cause1": "germs and microbes",
  "effect1": "sickness",
  "causality_phrase1": "cause"}
sent_cause_effect: [('The presentation uses animation to show how  germs  and  microbes  cause  sickness , and outlines simple preventive measures.', 'germs and microbes', 'sickness')]
i: 74
out :  {"cause": "the discharge of accumulated energy along geologic faults", "effect": "earthquakes", "causality_phrase": "are caused by"}
empty
sent_cause_effect: [(' Earthquakes  are caused by  the discharge  of accumulated along geologic faults.', 'the discharge of accumulated energy along geologic faults', 'earthquakes')]
i: 75
out :  {
  "cause1": "the quake",
  "effect1": "a fire",
  "causality_phrase1": "after",
  "cause2": "the quake",
  "effect2": "hundreds of people were injured",
  "causality_phrase2": "after"}
sent_cause_effect: [('Three people had been killed in  a fire  after  the quake  and hundreds of people were injured.', 'the quake', 'a fire'), ('Three people had been killed in  a

out :  {
  "cause": "the bacterial toxin",
  "effect": "muscle paralysis",
  "causality_phrase": "caused by"
}
empty
sent_cause_effect: [('These are all symptoms of  the muscle paralysis  caused by  the bacterial toxin .', 'the bacterial toxin', 'muscle paralysis')]
i: 99
out :  {
  "cause": "the fungus",
  "effect": "irritation",
  "causality_phrase": "caused by"
}
empty
sent_cause_effect: [('Getting help has reduced the time to reverse  all the irritation  that had been caused by  the fungus .', 'the fungus', 'irritation')]
i: 100
out :  {
  "cause": "the earthquake",
  "effect": "a large tsunami triggered",
  "causality_phrase": "triggered by"
}
empty
sent_cause_effect: [(' A large tsunami  triggered by  the earthquake  spread outward from off the Sumatran coast.', 'the earthquake', 'a large tsunami triggered')]
i: 101
out :  {
  "cause": "the burning of coal",
  "effect": "the light-colored tree trunks became darker due to soot",
  "causality_phrase": "due to"
}
empty
sent_cause_ef

out :  {
  "cause": "the atomic bomb explosion",
  "effect": "radiation",
  "causality_phrase": "from"
}
empty
sent_cause_effect: [(' The radiation  from  the atomic bomb explosion  is a typical acute radiation.', 'the atomic bomb explosion', 'radiation')]
i: 125
out :  {
  "cause": "the earthquake",
  "effect": "the largest landslide",
  "causality_phrase": "triggered by"
}
empty
sent_cause_effect: [(' The largest landslide  triggered by  the earthquake  is located approximately 32 kilometers southeast of Muzafarrabad in a tributary valley of the Jhelum River.', 'the earthquake', 'the largest landslide')]
i: 126
out :  {
  "cause": "insecurity",
  "effect": "violence",
  "causality_phrase": "thus causes"
}
empty
sent_cause_effect: [('Priority must be given to efforts to combat insecurity and thus  poverty , major cause of  violence  most often experienced by poorest.', 'insecurity', 'violence')]
i: 127
out :  {
  "cause": "The accident",
  "effect": "a major traffic snarl",
  "causali

out :  {
  "cause": "a virus",
  "effect": "Plantar warts",
  "causality_phrase": "are caused by"
}
empty
sent_cause_effect: [(' Plantar warts  are caused by  a virus  that infects the outer layer of skin on the soles of the feet.', 'a virus', 'Plantar warts')]
i: 151
out :  {
  "cause": "the switching process",
  "effect": "the interference",
  "causality_phrase": "triggered by"
}
empty
sent_cause_effect: [('Method according to claim 1, characterized in that the time duration of the quality reduction is limited at least to the duration of  the interference  triggered by  the switching process .', 'the switching process', 'the interference')]
i: 152
out :  {"cause": "", "effect": "proof of structure of the indole components of the seeds", "causality_phrase": "directed toward"}
empty
sent_cause_effect: [(' These chemical studies  were directed toward  proof  of structure of the indole components of the seeds.', '', 'proof of structure of the indole components of the seeds')]
i: 153
out 

out :  {
  "cause": "giving",
  "effect": "moral strength and happiness"
}
empty
out :  {
  "cause": "a strong offshore earthquake",
  "effect": "a tsunami is generated",
  "causality_phrase": "is generated by"
}
empty
sent_cause_effect: [('When  a tsunami  is generated by  a strong offshore earthquake , its first waves would reach the outer coast minutes after the ground stops shaking.', 'a strong offshore earthquake', 'a tsunami is generated')]
i: 175
out :  {
  "cause": "a homogeneous group of keratinophilic fungi termed the dermatophytes",
  "effect": "the cutaneous mycoses",
  "causality_phrase": "are caused by"
}
empty
sent_cause_effect: [(' The cutaneous mycoses  are caused by a homogeneous group of  keratinophilic fungi  termed the dermatophytes.', 'a homogeneous group of keratinophilic fungi termed the dermatophytes', 'the cutaneous mycoses')]
i: 176
out :  {
  "cause1": "the Prestige oil spill",
  "effect1": "the ecological catastrophe",
  "causality_phrase1": "caused by"}
se

In [8]:
len(filtered_rows)

191

In [9]:
#filtered_rows = filtered_rows.drop('GPT_causality_label', axis = 1)
#filtered_rows = filtered_rows.drop('patternmatcher_label', axis = 1)
len(fewshotfound_inputsentence_index)

0

In [10]:
filtered_rows['sent_cause_effect'] = sent_cause_effect_list
filtered_rows['causality_phrase'] = causality_phrase_list
filtered_rows

,Unnamed: 0.1,Unnamed: 0,sentence,embedings,sent_cause_effect,causality_phrase
0,0,0,Using solar electricity instead of conventiona...,"[0.030286945402622223, 0.00630235718563199, -0...",[(Using solar electricity instead of conventio...,"[helps to reduce, that causes]"
1,1,1,"Thus, evaluating capital punishment as a form ...","[-0.011410091072320938, 0.005278998054563999, ...","[(Thus, evaluating capital punishment as a for...",[that result in]
2,2,2,Sudden death from inhalation of petroleum ...,"[-0.0007025353843346238, 0.00263791810721159, ...",[( Sudden death from inhalation of petroleu...,[from]
3,3,3,Stimulation of the posterior horn of the med...,"[-0.03430342674255371, 0.01450358610600233, 0....",[( Stimulation of the posterior horn of the m...,[produced]
4,4,4,Sip the tea slowly to reduce stomach pain fr...,"[0.005228457972407341, -0.011144616641104221, ...",[(Sip the tea slowly to reduce stomach pain ...,[from]
...,...,...,...,...,...,...
186,186,186,The disruption has been caused by work bei...,"[-0.019408993422985077, -0.003347919089719653,...",[( The disruption has been caused by work b...,[has been caused by]
187,187,187,"Most illnesses, including colds and flu , c...","[0.007080608978867531, -0.004280312918126583, ...","[(Most illnesses, including colds and flu ,...","[cause, that also]"
188,188,188,Every child and young person seen at Starship ...,"[0.019172143191099167, 0.013259614817798138, 0...",[(Every child and young person seen at Starshi...,[had]
189,189,189,The movie gives the inaccurate impression ...,"[-0.04086083546280861, -0.011357069946825504, ...",[( The movie gives the inaccurate impression...,[gives]


In [11]:
#SCITE

pm_DB = pd.read_csv('data/in/RAG/c_e_Extraction/SCITE_CE_extract_INPUTkeys.csv')
pm_DB['keys'] = pm_DB['keys'].apply(eval)



C_t = pm_DB['keys'].to_list()


C_p = sent_cause_effect_list

In [12]:
len(C_t)

296

In [13]:
C_p = [tup for sublist in C_p for tup in sublist]
len(C_p[1])


3

In [14]:
def get_score(true_phrase, pred_phrase ):
    scores=[]
    p = true_phrase
    window_size = len(p)

        # Slide through input_sentence with a window size of 8
    if (pred_phrase) =="":
        print('no pred_phrase')
        return 0
    else: 
        for i in range(len(pred_phrase) - window_size + 1):
            window = pred_phrase[i:i + window_size]
            score = fuzz.ratio(window, p)
            scores.append(score)
        if len(scores) ==0:
            print('prediction length lower', pred_phrase, 'p:', p)
            return 0
        else:
            return max(scores)

In [15]:
len(C_p)

188

In [16]:
from fuzzywuzzy import fuzz

prediction= []
pred_label = []
for i in range(len(C_p)):
    #print(i)
    labeled = 0   
    if (C_p[i] != []):
        if len(C_p[i]) == 3:           
            
            sent = C_p[i][0]
            ####print('sent: ',sent )
            cause_pred = C_p[i][1]
            effect_pred = C_p[i][2]
            
            for i_t in range(len(C_t)):
                #get index of all tuples which match the sent and create sent_found = []
                # List to store indices
                sent_found = []

                # Check if sent is in the C_t at index 0
                for index, (sentence, _, _) in enumerate(C_t):
                    if sentence == sent:
                        sent_found.append(index)
            ####print('sent_found: ', sent_found)
            
            
            #iterate through sent_found and pass each cause_t and effect_t to get score: 
            for ind in sent_found:
                cause_t = C_t[ind][1]
                effect_t = C_t[ind][2]
                pred_c=0
                pred_e=0
                ##print('cause_t: ', cause_t)
                ##print('effect_t: ', effect_t)
                ##print('cause_pred: ', cause_pred)
                ##print('effect_pred: ', effect_pred)
                
                score = get_score(cause_t.lower(), cause_pred.lower())################ non -medcause
                    #score = fuzz.ratio(cause_pred, cause_t)################medcause
                if score >90:################90 non -medcause
                    pred_c = 1
                else:
                    pred_c=0
                    


                score = get_score(effect_t.lower(), effect_pred.lower())################non - medcause
                    #score = fuzz.ratio(effect_pred, effect_t)################medcause

                if score >90: ################90 non -medcause
                    pred_e = 1
                else:
                    pred_e=0



                if ((pred_e ==1) & (pred_c ==  1)):
                    prediction.append(1)
                    
                    labeled = 1
                    #pred_label.append(labeled)
                    #print('out:', 1)
    
    
    
    
    pred_label.append(labeled)
                    
    print(labeled, i)



                
    

 

0 0
1 1
1 2
1 3
1 4
0 5
1 6
1 7
1 8
0 9
prediction length lower wind p: smoke
1 10
0 11
1 12
prediction length lower suffering p: the suffering
prediction length lower suffering p: the suffering
prediction length lower suffering p: the suffering
prediction length lower suffering p: the suffering
0 13
prediction length lower headache p: a headache
0 14
prediction length lower accolade p: the accolade
0 15
0 16
1 17
1 18
prediction length lower steam p: the steam
prediction length lower steam p: a backpressure
prediction length lower rupture disc opening p: the rupture disc opening
0 19
1 20
prediction length lower discomfort p: enough discomfort
0 21
prediction length lower evacuation p: an evacuation
0 22
1 23
prediction length lower irritation p: skin irritation
prediction length lower the sun p: environmental pollutants
prediction length lower irritation p: skin irritation
0 24
prediction length lower inflammation p: skin irritation
prediction length lower inflammation p: skin irrita

In [17]:
prec=sum(prediction)/len(C_p)
rec = sum(prediction)/len(C_t)
F1 = 2 *(prec * rec)/(prec + rec)
print('F1: ', F1)
print('prec: ', prec)
print('rec: ', rec)


F1:  0.6239669421487603
prec:  0.8031914893617021
rec:  0.5101351351351351


In [18]:
len(pred_label)

188

In [19]:
len(C_p)

188

In [20]:
SCITE_df = pd.DataFrame({'pred_label': pred_label, 'C_p' : C_p})
SCITE_df

,pred_label,C_p
0,0,(Using solar electricity instead of convention...
1,1,(Using solar electricity instead of convention...
2,1,"(Thus, evaluating capital punishment as a form..."
3,1,( Sudden death from inhalation of petroleum...
4,1,( Stimulation of the posterior horn of the me...
...,...,...
183,1,"(Most illnesses, including colds and flu , ..."
184,0,"(Most illnesses, including colds and flu , ..."
185,0,(Every child and young person seen at Starship...
186,1,( The movie gives the inaccurate impression ...


In [21]:
#filtered_rows.to_csv('data/out/RAG_C_E_CP_Alzeimers.csv')#data/out/RAG_causality/semeval/semeval_
#filtered_rows.to_csv('data/out/RAG_causality/semeval/fewshot_sem_CE_detection.csv')#data/out/RAG_causality/semeval/semeval_

#fewshot_semeval_CE_detection
#SCITE_df.to_csv('data/out/RAG_causality/task2/gemma/KNN_SCITE_CE_extract.csv')
#filtered_rows.to_csv('data/out/RAG_causality/task2/GPT4/zero_sem_CE_extract.csv')#zero,rand,RAG|sem,ADE,SCITE


In [22]:
#filtered_rows= filtered_rows[filtered_rows['predictions'] == 0]
#filtered_rows.to_csv('data/out/RAG_causality/semeval/zero_sem_RAG_C_E_pred0.csv')#data/out/RAG_causality/semeval/semeval_
